In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [2]:
%%R 

suppressPackageStartupMessages({
    library(cowplot)
    library(patchwork)
    library(tidyverse)
    theme_set(theme_cowplot())
})

options(dplyr.width = Inf)

In [2]:
# Set seed with numpy
np.random.seed(123)

# Generate two example time-series of length 16
TS_array = np.random.randn(16, 2)

In [4]:
%%R -i TS_array

# Line plots
as.data.frame(TS_array) %>% 
    mutate(Timepoint = 1:16) %>%
    rename("TS_1" = V1, "TS_2" = V2) %>%
    pivot_longer(cols = c(TS_1, TS_2), names_to = "TS", values_to = "Value") %>%
    ggplot(aes(x = Timepoint, y = Value, color = TS)) +
    geom_line(linewidth=1.25) +
    facet_grid(TS ~ ., scales = "free_y") +
    theme_void() +
    scale_color_manual(values=c("#a663ce", "#67913c")) +
    theme(legend.position="none",
          axis.title=element_blank(),
          strip.text = element_blank())
ggsave("example_time_series_lines.svg", width=3.5, height=1, units="in", dpi=300)

In [6]:
%%R -i TS_array

# Take the first column of TS_array as a data.frame
TS1_heatmap <- as.data.frame(TS_array) %>%
    ggplot(aes(x = 1:16, y = 0, fill=V1)) +
    geom_tile() +
    theme_void() +
    scale_fill_gradient(low="#E8D7F2", high="#a663ce") +
    theme(legend.position="none",
          axis.title=element_blank(),
          strip.text = element_blank())
TS2_heatmap <- as.data.frame(TS_array) %>%
    ggplot(aes(x = 1:16, y = 0, fill=V2)) +
    geom_tile() +
    theme_void() +
    scale_fill_gradient(low="#D9E9C9", high="#67913c") +
    theme(legend.position="none",
          axis.title=element_blank(),
          strip.text = element_blank())

TS1_heatmap / TS2_heatmap
ggsave("example_time_series_vectors.svg", width=3, height=2, units="in", dpi=300)

### Checking TLMI symmetry

In [4]:
import numpy as np
import random

# Set seed with numpy
np.random.seed(123)

M = 2 # 5 independent processes
T = 200 # 500 samples per process

dataset = np.random.randn(M,T) # generate our multivariate time-series

In [17]:
from pyspi.statistics import infotheory
TLMI = infotheory.TimeLaggedMutualInfo()

X=dataset[0,:]
Y=dataset[1,:]

# z-score each array
X_z = (X - np.mean(X)) / np.std(X)
Y_z = (Y - np.mean(Y)) / np.std(Y)

TLMI_XY = TLMI.bivariate(data=X, data2=Y)
TLMI_YX = TLMI.bivariate(data=Y, data2=X)

print(f"TLMI(X->Y) = {TLMI_XY}, TLMI(Y->X) = {TLMI_YX}")

Normalising the dataset...

Normalising the dataset...

TLMI(X->Y) = 0.00046859326511109665, TLMI(Y->X) = 0.0007813702420392038


In [ ]:
from pyspi.calculator import Calculator
calc_tlmi = Calculator(dataset=dataset, configfile="TLMI_config.yaml") # instantiate the calculator object
calc_tlmi.compute()
print(calc_tlmi.table['tlmi_gaussian'])

In [ ]:
calc = Calculator(dataset=dataset) # instantiate the calculator object
calc.compute()

calc_res = deepcopy(calc.table)

In [20]:
calc_res = calc.table
# Iterate over each SPI
calc_res.columns = calc_res.columns.to_flat_index()

# Convert index to column
calc_res.reset_index(level=0, inplace=True)

# Rename index as first brain region
calc_res = calc_res.rename(columns={"index": "TS_from"})

# Pivot data from wide to long
calc_res_long = pd.melt(calc_res, id_vars="TS_from")
calc_res_long['SPI'], calc_res_long['TS_to'] = calc_res_long.variable.str

# Remove variable column
calc_res_long = calc_res_long.drop("variable", 1)


Columnar iteration over characters will be deprecated in future releases.
In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.


### 1. Are there any SPIs that are labelled as 'directed' but have the same values in both directions?

### 2. How many SPIs have values that differ slightly when you call them manually versus through the calculator object?